In [ ]:
!pip install --upgrade datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cc100", lang = "ar", split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

## The below is basically a copy of all the helper functions in Rasmi2.ipynb

In [ ]:
letter_dict = {
    "ا": "ا",
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ب": "ب",
    "ت": "ب",
    "ث": "ب",
    "ج": "ح",
    "ح": "ح",
    "خ": "ح",
    "د": "د",
    "ذ": "د",
    "ر": "ر",
    "ز": "ر",
    "س": "س",
    "ش": "س",
    "ص": "ص",
    "ض": "ص",
    "ط": "ط",
    "ظ": "ط",
    "ع": "ع",
    "غ": "ع",
    "ف": "ف",
    "ق": "ف",
    "ك": "ك",
    "ل": "ل",
    "م": "م",
    "ن": "ن",
    "و": "و",
    "ؤ": "و",
    "ه": "ه",
    "ة": "ه",
    "ي": "ى",
    "ى": "ى",
    "ئ": "ى",
}

In [ ]:
letter_tuple = (
    "ـ"
    "ا"
    "ل"
    "ن"
    "م"
    "ي"
    "و"
    "ه"
    "ب"
    "ر"
    "ع"
    "أ"
    "ف"
    "ق"
    "د"
    "ت"
    "س"
    "ك"
    "ح"
    "ة"
    "ى"
    "ج"
    "ص"
    "إ"
    "ذ"
    "ث"
    "خ"
    "ش"
    "ز"
    "ط"
    "ض"
    "غ"
    "ء"
    "ئ"
    "ظ"
    "آ"
    "ؤ"
)

In [ ]:
sample_text2 = """لكن لا بد أن أوضح لك أن كل هذه الأفكار المغلوطة حول استنكار  النشوة وتمجيد الألم نشأت بالفعل، وسأعرض لك التفاصيل لتكتشف حقيقة وأساس تلك السعادة البشرية، فلا أحد يرفض أو يكره أو يتجنب الشعور بالسعادة، ولكن بفضل هؤلاء الأشخاص الذين لا يدركون بأن السعادة لا بد أن نستشعرها بصورة أكثر عقلانية ومنطقية فيعرضهم هذا لمواجهة الظروف الأليمة، وأكرر بأنه لا يوجد من يرغب في الحب ونيل المنال ويتلذذ بالآلام، الألم هو الألم ولكن نتيجة لظروف ما قد تكمن السعاده فيما نتحمله من كد وأسي.

و سأعرض مثال حي لهذا، من منا لم يتحمل جهد بدني شاق إلا من أجل الحصول على ميزة أو فائدة؟ ولكن من لديه الحق أن ينتقد شخص ما أراد أن يشعر بالسعادة التي لا تشوبها عواقب أليمة أو آخر أراد أن يتجنب الألم الذي ربما تنجم عنه بعض المتعة ؟
علي الجانب الآخر نشجب ونستنكر هؤلاء الرجال المفتونون بنشوة اللحظة الهائمون في رغباتهم فلا يدركون ما يعقبها من الألم والأسي المحتم، واللوم كذلك يشمل هؤلاء الذين أخفقوا في واجباتهم نتيجة لضعف إرادتهم فيتساوي مع هؤلاء الذين يتجنبون وينأون عن تحمل الكدح والألم ."""

In [ ]:
divisor = 1000000000000000.0

In [ ]:
input_list = list()
output_list = list()

In [ ]:
def number_cruncher(word):
  pos = len(word) - 1
  exp = 0
  num = 0.0

  while pos >= 0:
    num += letter_tuple.index(word[pos]) * 37 ** exp
    pos -= 1
    exp += 1

  num = round(num / divisor, 15)
  return num

In [ ]:
def generate_letter_codes(word, aList):

  if len(word) > 9:
    aList.append(number_cruncher(word[0:9]))
    generate_letter_codes(word[9:], aList)
  else:
    aList.append(number_cruncher(word + "ـ"))

In [ ]:
def parse_text(text):
  text = text.strip()
  index = 0

  while index < len(text):
    space = text.find(" ", index)

    if space == -1: break

    output_string = text[index:space]
    input_string = ""

    for y in output_string:
      if y in letter_dict:
        input_string = input_string + letter_dict[y]
      else:
        output_string = output_string.replace(y, "")

    generate_letter_codes(input_string, input_list)
    generate_letter_codes(output_string, output_list)

    index = space
    while text[index] == " ":
      index += 1

In [ ]:
def decoder(num):
  word = ""
  num = round(num * divisor)

  while num > 36:
    word = letter_tuple[int(num % 37)] + word
    num = np.trunc(num / 37)

  word = letter_tuple[int(num)] + word

  return word

In [ ]:
def list_decoder(aList):
  txt = ""
  for x in aList:
    y = decoder(x)
    if y[len(y) - 1] == "ـ":
      txt += y[0:len(y) - 1] + " "
    else:
      txt += y

  return txt

In [ ]:
sample_text2 = """لكن لا بد أن أوضح لك أن كل هذه الأفكار المغلوطة حول استنكار  النشوة وتمجيد الألم نشأت بالفعل، وسأعرض لك التفاصيل لتكتشف حقيقة وأساس تلك السعادة البشرية، فلا أحد يرفض أو يكره أو يتجنب الشعور بالسعادة، ولكن بفضل هؤلاء الأشخاص الذين لا يدركون بأن السعادة لا بد أن نستشعرها بصورة أكثر عقلانية ومنطقية فيعرضهم هذا لمواجهة الظروف الأليمة، وأكرر بأنه لا يوجد من يرغب في الحب ونيل المنال ويتلذذ بالآلام، الألم هو الألم ولكن نتيجة لظروف ما قد تكمن السعاده فيما نتحمله من كد وأسي.

و سأعرض مثال حي لهذا، من منا لم يتحمل جهد بدني شاق إلا من أجل الحصول على ميزة أو فائدة؟ ولكن من لديه الحق أن ينتقد شخص ما أراد أن يشعر بالسعادة التي لا تشوبها عواقب أليمة أو آخر أراد أن يتجنب الألم الذي ربما تنجم عنه بعض المتعة ؟
علي الجانب الآخر نشجب ونستنكر هؤلاء الرجال المفتونون بنشوة اللحظة الهائمون في رغباتهم فلا يدركون ما يعقبها من الألم والأسي المحتم، واللوم كذلك يشمل هؤلاء الذين أخفقوا في واجباتهم نتيجة لضعف إرادتهم فيتساوي مع هؤلاء الذين يتجنبون وينأون عن تحمل الكدح والألم ."""